# Designing and Analyzing the Results of an A/B Test

## Experiment Overview

Udacity is an online education company.  In this experiment, they tested a change to their course sign-up.  After clicking the “Start free trial” button, users were prompted to state how many hours per week they could dedicate to the course.  If a user entered fewer than five hours then they were told that they would need to devote more time than that.  In addition, Udacity proffered the information that the course materials could be accessed for free if the user was unsure of their commitment.  If the user entered more than five hours then they were directed to enter their credit card information and start their 14-day free trial.

<img src = "course-sign-up.png">
> Screen capture of the experimental treatment

In creating this change, Udacity was interested in determining if this “warning” would help users to better understand the commitment required of the course, and hence decrease future frustration.  Udacity hopes this will lead more users to finish their free trial and continue to remain enrolled into the first pay period.